In [1]:
import pandas as pd
import requests
import json
import numpy as np

### Get API data from themoviedb.id

In [2]:
movie_api = "https://api.themoviedb.org/3/movie/{}?"
api_key  = "###"

In [3]:
movies = []

In [4]:
for i in range(0,10):
    url = movie_api.format(i) + api_key
    data = requests.get(url)
    if data.status_code ==404:
        continue
    else:
        movie = data.json()
        movies.append(movie)

In [5]:
df = pd.DataFrame.from_dict(data=movies)

In [7]:
df.to_csv('movies_unclean.csv', index= False)

In [48]:
df = pd.read_csv('movies_unclean.csv', low_memory=False)

In [49]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [35]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

### Dropping unwanted columns

In [5]:
df.drop(columns=['adult', 'imdb_id', 'video','original_title', 'homepage'],inplace=True)

### Evaluate stringified JSON columns 

In [6]:
df.genres[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [7]:
df['belongs_to_collection'][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [8]:
df['production_countries'][0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

Some columns with stringified JSON : genres, belongs_to_collection, production_countries, production_companies, spoken_languages
Because json requires the dict to be '{"x":1, "y":2}' and not the other way around

In [36]:
stringified_json = ['genres', 'belongs_to_collection', 'production_countries', 'production_companies', 'spoken_languages']

In [37]:
import ast

The problem with ast is that only accept string as an input. But if we have a null value (NaN) it will automatically throw an error


So we need first to check if the variable is a string data type

In [38]:
for i in stringified_json:
    df[i] = df[i].apply(lambda x:ast.literal_eval(x) if isinstance(x,str) else x)

In [39]:
df['belongs_to_collection'][0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

### Flatten the nested columns

In [40]:
df['belongs_to_collection']

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object

In [41]:
df['belongs_to_collection'][0]

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

The json columns are still in nested columns, so we need to flatten these columns

We don't need all the columns yet only the necessary one, for example: belongs_to_collection we need only to extract the 'name' only. the others are not necessary

But now we need first to distinguished the row with NaN

In [43]:
df['belongs_to_collection'][0]['name']

'Toy Story Collection'

In [45]:
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x:x['name'] if isinstance(x,dict) else x)

In [17]:
df['genres'][0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

For df['genres'] there are more than one 'name' so we need to extract them and join all the names

In [18]:
df['genres'] = df['genres'].apply(lambda x: ','.join(i['name'] for i in x))

In [19]:
df['genres'].apply(lambda x:0 if len(x)==0 else 1).value_counts()

1    43024
0     2442
Name: genres, dtype: int64

In [20]:
df[df['genres']==''].shape

(2442, 19)

So there are 'missing value' in the df['genres']

In [21]:
df['genres'] = df['genres'].apply(lambda x:np.NaN if len(x)==0 else x)

In [22]:
df['genres'].isnull().sum()

2442

In [23]:
df['production_countries'][0]

[{'iso_3166_1': 'US', 'name': 'United States of America'}]

For df['production_countries'] we need to extract the 'name' only

In [24]:
df['production_countries'].apply(lambda x:len(x) if isinstance(x,list) else np.NaN).value_counts().head()

1.0    32151
0.0     6282
2.0     4875
3.0     1469
4.0      462
Name: production_countries, dtype: int64

Apparently there are more than one country for df['production_countries'] so we have to join all of them again

In [25]:
df['production_countries'] = df['production_countries'].apply(lambda x: ','.join(i['name'] for i in x) if  isinstance(x,list) else np.NaN)

In [26]:
df['production_companies'][0]

[{'name': 'Pixar Animation Studios', 'id': 3}]

In [27]:
df['production_companies'].apply(lambda x:len(x) if isinstance(x,list) else np.NaN).value_counts().head()

1.0    16577
0.0    11875
2.0     7963
3.0     4716
4.0     1948
Name: production_companies, dtype: int64

It goes also for the df['production_companies']

In [28]:
df['production_companies'] = df['production_companies'].apply(lambda x: ','.join(i['name'] for i in x) if  isinstance(x,list) else np.NaN)

In [29]:
df['spoken_languages'][0]

[{'iso_639_1': 'en', 'name': 'English'}]

In [30]:
df['spoken_languages'].apply(lambda x:len(x) if isinstance(x,list) else np.NaN).value_counts().head()

1.0    33736
2.0     5371
0.0     3829
3.0     1705
4.0      550
Name: spoken_languages, dtype: int64

And df['spoken_languages'] too

In [31]:
df['spoken_languages'] = df['spoken_languages'].apply(lambda x: ','.join(i['name'] for i in x) if  isinstance(x,list) else np.NaN)

In [32]:
df.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,Toy Story Collection,30000000,"Animation,Comedy,Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"Adventure,Fantasy,Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,1995-12-15,262797249.0,104.0,"English,Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0


### Cleaning Numerical Columns

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 43024 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   poster_path            45080 non-null  object 
 8   production_companies   45460 non-null  object 
 9   production_countries   45460 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue                45460 non-null  float64
 12  runtime                45203 non-null  float64
 13  spoken_languages       45460 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

There are four columns that already as float datatype: revenue, runtime, vote_average, vote_count

Column 'budget', 'id', and 'popularity' are numerical data but still in object datatype

In [39]:
df.loc[19730, 'budget']

'/ff9qCepilowshEtG2GYWwzt2bs4.jpg'

There are errors in the provided data for 'budget' column. So we need to pass the invalid data as NaN

In [40]:
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

In [41]:
df['budget'].dtype

dtype('float64')

In [42]:
df['budget'].value_counts(dropna=False)

0.0           36573
5000000.0       286
10000000.0      259
20000000.0      243
2000000.0       242
              ...  
9750000.0         1
7275000.0         1
78146652.0        1
280.0             1
1254040.0         1
Name: budget, Length: 1224, dtype: int64

Replace 0 value with NaN

In [43]:
df['budget'].replace(0, np.NaN, inplace=True)

Change the currency from usd to million usd

In [44]:
df['budget'] = df['budget']/1000000

In [45]:
df.loc[19730, 'id']

'1997-08-20'

Errors on the 'id' data

In [46]:
df['id'] = pd.to_numeric(df['id'], errors='coerce')

In [47]:
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')

In [48]:
df['revenue'] = df['revenue']/1000000

In [49]:
df.rename(columns={'budget':'budget_mio', 'revenue':'revenue_mio'}, inplace=True)

In [50]:
df.loc[df['revenue_mio']==0, 'revenue_mio'].shape

(38052,)

So there are a lot of 0 USD revenue. We will replace this with NaN 

In [51]:
df['revenue_mio'] = df['revenue_mio'].replace(0, np.NaN)

In [52]:
df.loc[df['runtime']==0, 'runtime'].shape

(1558,)

In [53]:
df['runtime'] = df['runtime'].replace(0, np.NaN)

In [54]:
df['vote_count'].value_counts().head()

1.0    3264
2.0    3132
0.0    2899
3.0    2787
4.0    2480
Name: vote_count, dtype: int64

In [55]:
df['vote_average'].value_counts().head()

0.0    2998
6.0    2468
5.0    2001
7.0    1886
6.5    1722
Name: vote_average, dtype: int64

If the vote_count is 0, we don't know the vote_average. So we can replace the vote_average to NaN value if the vote_count is 0

In [56]:
df.loc[df['vote_count']==0, 'vote_average'] = np.NaN

### Handling Datetime Values

We have only one datetime column: 'release_date'

In [57]:
df['release_date'].dtype

dtype('O')

In [58]:
df['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [59]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

### Handling Text/Str values

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4494 non-null   object        
 1   budget_mio             8890 non-null   float64       
 2   genres                 43024 non-null  object        
 3   id                     45463 non-null  float64       
 4   original_language      45455 non-null  object        
 5   overview               44512 non-null  object        
 6   popularity             45460 non-null  float64       
 7   poster_path            45080 non-null  object        
 8   production_companies   45460 non-null  object        
 9   production_countries   45460 non-null  object        
 10  release_date           45376 non-null  datetime64[ns]
 11  revenue_mio            7408 non-null   float64       
 12  runtime                43645 non-null  float64       
 13  s

In [61]:
df['original_language'].value_counts()

en       32269
fr        2438
it        1529
ja        1350
de        1080
         ...  
zu           1
qu           1
104.0        1
la           1
si           1
Name: original_language, Length: 92, dtype: int64

In [62]:
df['original_language'].unique()

array(['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'cn', 'ar', 'es', 'ru',
       'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy',
       'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'xx', 'mk', 'bo',
       'ca', 'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo',
       'ka', 'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv',
       'ta', 'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms',
       'sq', nan, '104.0', 'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo',
       'ay', 'kn', 'eu', 'ne', 'pa', 'ky', 'gl', '68.0', 'uz', 'sm', 'mt',
       '82.0', 'hy', 'iu', 'lb', 'si'], dtype=object)

So there are number as string. We can change those numbers to np.NaN and also the 'nan'

In [63]:
df['original_language'] = df['original_language'].replace(['82.0', '104.0', '68.0', 'nan'], np.NaN)

In [64]:
df['overview'].value_counts().head()

No overview found.                                            133
No Overview                                                     7
                                                                5
No movie overview available.                                    3
A few funny little novels about different aspects of life.      3
Name: overview, dtype: int64

There are three NaN values: 'No overview found', 'No Overview' and 'No movie overview available.' and ''

In [65]:
overview_nan = ['No overview found.', 'No Overview', 'No movie overview available.', ' '] 

In [66]:
df['overview'] = df['overview'].apply(lambda x:np.NaN if x in(overview_nan) else x)

In [67]:
df['tagline'].value_counts().head()

Based on a true story.           7
Be careful what you wish for.    4
Trust no one.                    4
-                                4
Classic Albums                   3
Name: tagline, dtype: int64

In [68]:
df.loc[df['tagline']=='-', 'tagline'] = np.NaN

### Removing Duplicates

In [75]:
df[df.duplicated(keep=False)].sort_values(by='id').head()

,belongs_to_collection,budget_mio,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_mio,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
7345,NaN,NaN,"Crime,Drama,Thriller",5511.0,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,"Fida cinematografica,Compagnie Industrielle et...","France,Italy",1967-10-25,0.039481,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
9165,NaN,NaN,"Crime,Drama,Thriller",5511.0,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,"Fida cinematografica,Compagnie Industrielle et...","France,Italy",1967-10-25,0.039481,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
24844,NaN,NaN,"Comedy,Drama",11115.0,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,"Andertainment Group,Crescent City Pictures,Tag...",United States of America,2008-01-29,NaN,85.0,English,Released,NaN,Deal,5.2,22.0
14012,NaN,NaN,"Comedy,Drama",11115.0,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,"Andertainment Group,Crescent City Pictures,Tag...",United States of America,2008-01-29,NaN,85.0,English,Released,NaN,Deal,5.2,22.0
22151,NaN,NaN,"Action,Horror,Science Fiction",18440.0,en,When a comet strikes Earth and kicks up a clou...,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,,United States of America,2007-01-01,NaN,89.0,English,Released,NaN,Days of Darkness,5.0,5.0


There are duplicates regarding the movie id

In [82]:
df.drop_duplicates(subset='id', inplace=True)

### Handling missing values

In [84]:
df.isnull().sum()

belongs_to_collection    40945
budget_mio               36554
genres                    2442
id                           1
original_language           12
overview                  1102
popularity                   4
poster_path                386
production_companies         4
production_countries         4
release_date                88
revenue_mio              38036
runtime                   1819
spoken_languages             4
status                      85
tagline                  25037
title                        4
vote_average              2900
vote_count                   4
dtype: int64

In [90]:
df.notna().sum(axis=1).value_counts()

15    16799
16    13852
17     4738
18     3978
14     3166
13     1204
19     1135
12      385
11      144
10       25
9         4
4         3
5         1
dtype: int64

In [97]:
df.isna().sum(axis=1).value_counts()

4     16799
3     13852
2      4738
1      3978
5      3166
6      1204
0      1135
7       385
8       144
9        25
10        4
15        3
14        1
dtype: int64

We will drop the row with more than 9 missing values

In [98]:
df.dropna(axis=0, thresh=10, inplace=True)

Additionally we will drop the missing value found in 'id' and 'title' columns

In [99]:
df.dropna(subset=['id', 'title'],inplace=True)

In [100]:
df.isnull().sum()

belongs_to_collection    40939
budget_mio               36546
genres                    2438
id                           0
original_language           11
overview                  1098
popularity                   0
poster_path                379
production_companies         0
production_countries         0
release_date                80
revenue_mio              38028
runtime                   1811
spoken_languages             0
status                      81
tagline                  25029
title                        0
vote_average              2892
vote_count                   0
dtype: int64

### Handling the poster path

In [103]:
df['poster_path'][2]

'/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg'

In [108]:
base_poster_url = 'http://image.tmdb.org/t/p/w185/'

In [110]:
df['poster_path'] = base_poster_url+df['poster_path']

In [115]:
df.to_csv('movies_cln.csv', index=False)